# Escape the Mines

A poor miner is trapped in a mine and you have to help him to get out !

Only, the mine is all dark so you have to tell him where to go.

In this kata, you will have to implement a method solve(map, miner, exit) that has to return the path the miner must take to reach the exit as an array of moves, such as : ['up', 'down', 'right', 'left']. There are 4 possible moves, up, down, left and right, no diagonal.

map is a 2-dimensional array of boolean values, representing squares. false for walls, true for open squares (where the miner can walk). It will never be larger than 5 x 5. It is laid out as an array of columns. All columns will always be the same size, though not necessarily the same size as rows (in other words, maps can be rectangular). The map will never contain any loop, so there will always be only one possible path. The map may contain dead-ends though.

miner is the position of the miner at the start, as an object made of two zero-based integer properties, x and y. For example {x:0, y:0} would be the top-left corner.

exit is the position of the exit, in the same format as miner.

Note that the miner can't go outside the map, as it is a tunnel.

## Provided tests

## Solution

We first define a helper function that knows whether the point is in the map.

In [1]:
def on_map(minemap, miner):
    return (
        0 <= miner['x'] < len(minemap)
        and 0 <= miner['y'] < len(minemap[0])
    )

We use recursion to do a depth-first search of the paths in the mine.

In [2]:
def solve(minemap, miner, exit, path=None):
    path = path or []

    x_miner = miner['x']
    y_miner = miner['y']

    if miner == exit:
        return path

    # Dictionary converting word to displacement and vice-versa
    translate = {
        'up': (0, -1), (0, -1): 'up',
        'down': (0, 1), (0, 1): 'down',
        'left': (-1, 0), (-1, 0): 'left',
        'right': (1, 0), (1, 0): 'right',
    }

    # Identify where the miner is coming from
    if len(path) > 0:
        origin = translate[path[-1]]
        origin = tuple(-x for x in origin)
    else:
        origin = None

    for (dx, dy) in (-1, 0), (1, 0), (0, -1), (0, 1):
        if (
            # miner does not come from this direction
            (dx, dy) != origin
            # the target location is within the map
            and on_map(minemap, {'x': x_miner + dx, 'y': y_miner + dy})
            # the destination is not a wall
            and minemap[x_miner + dx][y_miner + dy]
        ):
            path_tmp = solve(
                minemap,
                {
                    'x': x_miner + dx,
                    'y': y_miner + dy
                },
                exit,
                path + [translate[(dx, dy)]]
            )
            if path_tmp[-1] != 'fail':
                return path_tmp

    return path + ['fail']

In [3]:
# A trivial map (1x1)
minemap = [[True]]

# Should return an empty array, since we're already at the goal
assert solve(minemap, {'x': 0, 'y': 0}, {'x': 0, 'y': 0}) == []

# A pretty simple map (2x2)
minemap = [[True, False],
           [True, True]]

# Should return the only correct move
assert solve(minemap, {'x': 0, 'y': 0}, {'x': 1, 'y': 0}) == ['right']

# Should return the only moves necessary
assert solve(minemap, {'x': 0, 'y': 0}, {'x': 1, 'y': 1}) == ['right', 'down']

# A linear map(1x4)
minemap = [[True], [True], [True], [True]]

# Should return a chain of moves to the right
assert solve(minemap, {'x': 0, 'y': 0}, {'x': 3, 'y': 0}) == [
    'right', 'right', 'right']

# Should return a chain of moves to the left
assert solve(minemap, {'x': 3, 'y': 0}, {'x': 0, 'y': 0}) == [
    'left', 'left', 'left']

# Should walk around an obstacle (3x3 map)
minemap = [[True, True, True],
           [False, False, True],
           [True, True, True]]

# Should return the right sequence of moves
assert solve(minemap, {'x': 0, 'y': 0}, {'x': 2, 'y': 0}) == [
    'down', 'down', 'right', 'right', 'up', 'up']